In this analysis, we will explore a dataset of google job ads. Those ads correspond either to the search string `machine learning engineer` or `data scientist`.

In [1]:
import os
import io
import re
import yaml

import pandas as pd
import numpy as np

from stemming import porter2

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import cross_validate
from sklearn.compose import ColumnTransformer

import matplotlib.pyplot as plt
import seaborn as sns

from google.cloud import storage

In [2]:
config_path = os.path.abspath(os.path.join(os.getcwd(), '..', 'config.yaml'))
with open(config_path, 'r') as config_file:
    config = yaml.load(config_file, Loader=yaml.SafeLoader)

In [3]:
%matplotlib inline

In [4]:
plt.style.use('fivethirtyeight')
plt.rcParams['figure.dpi']= 180

In [5]:
client = storage.Client(config['project'])
bucket = client.get_bucket(config['bucket'])
blob = bucket.blob(config['dest_file'])
file_content = io.BytesIO(blob.download_as_string())

ads = pd.read_csv(file_content)

/Users/ruslan.kozhuharov/venvs/tf/lib/python3.7/site-packages/google/auth/_default.py:69: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK. We recommend that most server applications use service accounts instead. If your application continues to use end user credentials from Cloud SDK, you might receive a "quota exceeded" or "API not enabled" error. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


In [6]:
ads.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 567 entries, 0 to 566
Data columns (total 7 columns):
id                  567 non-null object
job_descr           565 non-null object
location            561 non-null object
minimum_qual        565 non-null object
preferred_qual      565 non-null object
responsibilities    565 non-null object
title               567 non-null object
dtypes: object(7)
memory usage: 31.1+ KB


First, let us try using simple string operations to derive the additional information from the job title. Such information could be: seniority, function, etc.

In [7]:
ads.fillna('unknown', inplace=True)

In [8]:
ads[['role', 'department', 'area', 'other']] = ads.title.str.split(',', expand=True)

In [9]:
seniority_re = r'.*(intern|senior|lead|staff|principal|manager|head).*'
function_re = r'.*(engineer|scientist|science|consultant|architect|advocate|developer|analyst|specialist).*'

ads['seniority'] = ads.role.str.extract(pat=seniority_re, flags=re.I).fillna('L3-L4')[0].str.lower()
ads['function'] = (ads.role.str.extract(pat=function_re, flags=re.I)[0]
                   .str.lower().str.replace('science', 'scientist', flags=re.I))

In [10]:
ads[['title', 'role', 'department', 'area', 'seniority', 'function']].head()

,title,role,department,area,seniority,function
0,"Cloud AI Engineer, Professional Services",Cloud AI Engineer,Professional Services,None,l3-l4,engineer
1,"Cloud AI Engineer, Professional Services",Cloud AI Engineer,Professional Services,None,l3-l4,engineer
2,"Conversational AI Engineer, Google Cloud Profe...",Conversational AI Engineer,Google Cloud Professional Services,None,l3-l4,engineer
3,"Cloud AI Engineer, Professional Services",Cloud AI Engineer,Professional Services,None,l3-l4,engineer
4,"Data Scientist, Engineering",Data Scientist,Engineering,None,l3-l4,scientist


An interesting insight could be to see the most relevant terms from the qualifications and responsibilities with regards to the function. This could give us an isight into the keywords that one should emphasize on in their resume.

In [11]:
ads.seniority.value_counts()

l3-l4        377
staff         58
senior        39
intern        35
lead          27
manager       22
principal      8
head           1
Name: seniority, dtype: int64

In [12]:
ads.location.value_counts()

Mountain View          93
Sunnyvale              53
Bengaluru              36
Xindian District       23
Warsaw                 21
                       ..
Oslo                    1
Minneapolis             1
Zhubei, Zhubei City     1
Stockholm               1
Addison                 1
Name: location, Length: 68, dtype: int64

In [13]:
ads.function.value_counts()

engineer      380
architect      74
developer      18
scientist      16
consultant     11
analyst        11
specialist      3
advocate        2
Name: function, dtype: int64

In [14]:
to_model = ads[ads.function.isin(['engineer', 'scientist', 'architect'])].copy()

In [15]:
label_enc = LabelEncoder()
to_model['function_enc'] = label_enc.fit_transform(to_model.function.astype('str'))

In [16]:
kwargs = {'stop_words':'english',
          'ngram_range':(2, 5),
          'min_df':0.05,
          'max_df':0.95,
          'strip_accents':'unicode'}

text_cols = ['job_descr', 'minimum_qual', 'preferred_qual', 'responsibilities']

column_trans = ColumnTransformer([(f'tfidf_{c}', TfidfVectorizer(**kwargs), c) for c in text_cols])

multi_col_pipe = Pipeline([('multi_col_tfidf', column_trans),
                           ('naive_bayes_clf', MultinomialNB())])

In [17]:
column_trans.fit_transform(to_model[text_cols + ['location']])

<470x2507 sparse matrix of type '<class 'numpy.float64'>'
	with 172627 stored elements in Compressed Sparse Row format>

In [18]:
multi_col_pipe.fit(to_model[text_cols + ['location']], to_model.function_enc)

Pipeline(memory=None,
         steps=[('multi_col_tfidf',
                 ColumnTransformer(n_jobs=None, remainder='drop',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('tfidf_job_descr',
                                                  TfidfVectorizer(analyzer='word',
                                                                  binary=False,
                                                                  decode_error='strict',
                                                                  dtype=<class 'numpy.float64'>,
                                                                  encoding='utf-8',
                                                                  input='content',
                                                                  lowercase=True,
                                                                  max_df=0.95,
            

In [19]:
%time
scores = cross_validate(multi_col_pipe, to_model[text_cols], to_model.function_enc, cv=3, scoring='f1_weighted', return_train_score=True)

CPU times: user 2 µs, sys: 1 µs, total: 3 µs
Wall time: 6.2 µs


In [20]:
for k in scores.keys():
    print(f'{k}, mean: {scores[k].mean():.2f}, std: {scores[k].std():.2f}')

fit_time, mean: 0.53, std: 0.05
score_time, mean: 0.18, std: 0.03
test_score, mean: 0.78, std: 0.04
train_score, mean: 0.78, std: 0.01


In [21]:
importances_mcp = pd.DataFrame(multi_col_pipe['naive_bayes_clf'].feature_log_prob_.transpose(),
                               columns=label_enc.classes_,
                               index=multi_col_pipe['multi_col_tfidf'].get_feature_names())

In [22]:
col_prefix = re.compile('(' + '|'.join([f'tfidf_{c}__' for c in text_cols]) + ')(.*)')

In [23]:
top_n = 20
imp_terms = pd.DataFrame()

for l in label_enc.classes_:
    imp_label = importances_mcp[l].sort_values(ascending=False)[:top_n]
    to_append = [{'feature': col_prefix.match(feature_term)[1],
                  'term': col_prefix.match(feature_term)[2],
                  'impoortance': imp,
                  'label': l}
                 for feature_term, imp in zip(imp_label.index, imp_label.values)]
    imp_terms = imp_terms.append(to_append, ignore_index=True)

In [24]:
to_crosstab = imp_terms[imp_terms.label=='architect']

pd.crosstab(to_crosstab.term, to_crosstab.feature)

feature,tfidf_job_descr__,tfidf_minimum_qual__,tfidf_preferred_qual__,tfidf_responsibilities__
term,,,,
10 years experience,0,1,0,0
bachelor degree,0,1,0,0
cloud based,0,0,1,0
cloud computing,0,1,0,0
cloud technology,0,0,1,0
computer architecture,0,0,1,0
computer engineering,0,0,1,0
designing implementing,0,0,1,0
equivalent practical,0,1,0,0
